In [40]:
import torch
import numpy as np
from data import Task1Data, Task2Data, Task3Data
import os
import statsmodels.api as sm
import matplotlib.pyplot as plt


norm_dataset = Task1Data()
abide_dataset = Task2Data()
mci_dataset = Task3Data(no=2)
ad_dataset = Task3Data(no=1)

check_bins = 20
all_data = np.zeros((2,100,256,62))
name = 'res/search_param0.002_mom0.05/'
statis_dist = np.zeros((2,100,62, check_bins))
statis_std = np.zeros((2,100,62, 1))

for sex in range(2):
    for age in range(100):
        dat = np.load(f"{name}//sex-{sex}_age-{age}.npy")
        for bins in range(1,check_bins + 1):
            statis_dist[sex,age,:,bins-1] = np.quantile(dat[:,:62],bins/check_bins, axis=0)
        statis_std[sex,age,:,0] = dat.std(0).reshape(-1)[:62]
        all_data[sex,age] = dat[:,:62]

# use_feas = abide_dataset.ct_feas[:,:,0][...,None].repeat(check_bins,axis=2)
# use_age = abide_dataset.ages
# use_sex = abide_dataset.sexs
# use_dx = abide_dataset.dx


use_feas = ad_dataset.ct_feas[:,:,0][...,None].repeat(check_bins,axis=2)
use_age = ad_dataset.ages
use_sex = ad_dataset.sexs
use_dx = ad_dataset.dx


mine_feas = (use_feas - statis_dist[use_sex.astype(int),use_age.astype(int)] ) / statis_std[use_sex.astype(int),use_age.astype(int)]



(712, 62, 2) (712,) (712,)
(999, 62, 2) (999,) (999,)
(649, 62, 2) (649,) (649,)
Finding files: 2360/2360/2360
Finding files: 1031/1031/1031
Finding files: 712/712/712
380/332
Finding files: 604/604/604
272/332


In [47]:
use_feas = abide_dataset.ct_feas[:,:,0][...,None].repeat(check_bins,axis=2)
use_age = abide_dataset.ages
use_sex = abide_dataset.sexs
use_dx = abide_dataset.dx

mine_feas = (use_feas - statis_dist[use_sex.astype(int),use_age.astype(int)] ) / statis_std[use_sex.astype(int),use_age.astype(int)]


In [52]:
use_feas = ad_dataset.ct_feas[:,:,0][...,None].repeat(check_bins,axis=2)
use_age = ad_dataset.ages
use_sex = ad_dataset.sexs
use_dx = ad_dataset.dx


mine_feas = (use_feas - statis_dist[use_sex.astype(int),use_age.astype(int)] ) / statis_std[use_sex.astype(int),use_age.astype(int)]

In [60]:
use_s_score = mine_feas[...,12]
use_s_score.shape

(604, 62)

In [61]:
n_thres = 10
z_score_threshold = np.zeros((62,n_thres))

for roi in range(62):
    threshold_set = np.linspace(np.min(use_s_score[:,roi]), np.max(use_s_score[:,roi]), n_thres)
    z_score_threshold[roi] = threshold_set

In [62]:
from sklearn.metrics import accuracy_score

In [63]:
best_score=0
best_threshold_n=-1
for idx in range(n_thres):
    threshold = z_score_threshold[:, idx]
    labels_threshold = (use_s_score > threshold).astype(int).mean(1)
    labels_threshold[labels_threshold<0.5] = 0
    labels_threshold[labels_threshold>=0.5]=1
    score = accuracy_score(labels_threshold, use_dx)
    if score > best_score:
        best_score = score
        best_threshold = threshold
        best_threshold_n = idx

print(f"Best threshold accuracy: {best_score}, {best_threshold_n}")

Best threshold accuracy: 0.5496688741721855, 8


In [70]:
from sklearn.cluster import KMeans

# 尝试不同的k值
scores = []

kmeans = KMeans(n_clusters=2, random_state=40)
labels = kmeans.fit_predict(mine_feas.reshape((len(mine_feas), -1)))
# labels = kmeans.fit_predict(use_feas.reshape((len(use_feas), -1)))

z_scores = mine_feas[...,check_bins//2-1]

if z_scores[labels == 1].mean() > z_scores[labels == 0].mean():
    labels = 1 - labels
score = accuracy_score(labels, use_dx)
print(score)


0.7135761589403974


In [71]:
import pandas as pd

In [82]:
df = pd.read_csv("gpr.csv")

In [83]:
df

,fea,age,sex,male,group,GP_pred,GP_sigma,GP_residuals,GP_z
0,0.6278,39,1.0,0.0,0,0.592182,4.975210,0.035618,0.007159
1,0.5560,51,0.0,1.0,0,0.588990,4.976254,-0.032990,-0.006629
2,0.5738,69,0.0,1.0,0,0.582430,4.976252,-0.008630,-0.001734
3,0.6140,43,0.0,1.0,0,0.592091,4.976827,0.021909,0.004402
4,0.6516,21,0.0,1.0,0,0.613845,4.969589,0.037755,0.007597
...,...,...,...,...,...,...,...,...,...
4702,0.5900,72,0.0,1.0,3,0.584235,4.976233,0.005765,0.001158
4703,0.5754,70,0.0,1.0,3,0.582539,4.975806,-0.007139,-0.001435
4704,0.5662,76,1.0,0.0,3,0.582494,4.977103,-0.016294,-0.003274
4705,0.6164,76,1.0,0.0,3,0.582494,4.977103,0.033906,0.006812


In [75]:
df

,fea,age,sex,male,group,use_dx,LOESS_pred,LOESS_sigma,LOESS_residuals,LOESS_z,LOESS_rank
0,0.6278,39,1.0,0.0,0,0.0,0.601024,0.029849,0.026776,0.897056,0.0
1,0.5560,51,0.0,1.0,0,0.0,0.588364,0.030310,-0.032364,-1.067749,-1.0
2,0.5738,69,0.0,1.0,0,0.0,0.578858,0.033004,-0.005058,-0.153250,0.0
3,0.6140,43,0.0,1.0,0,0.0,0.596030,0.030085,0.017970,0.597291,0.0
4,0.6516,21,0.0,1.0,0,0.0,0.619792,0.031971,0.031808,0.994910,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4702,0.5900,72,0.0,1.0,3,0.0,0.577279,0.034843,0.012721,0.365099,0.0
4703,0.5754,70,0.0,1.0,3,0.0,0.578538,0.033462,-0.003138,-0.093775,0.0
4704,0.5662,76,1.0,0.0,3,0.0,0.575507,0.034073,-0.009307,-0.273160,0.0
4705,0.6164,76,1.0,0.0,3,0.0,0.575507,0.034073,0.040893,1.200134,1.0


In [76]:
set(df['use_dx'])

{0.0, 1.0}

In [81]:
df[df['group']==1]['use_dx'].describe()

count    1031.000000
mean        0.486906
std         0.500071
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: use_dx, dtype: float64

In [84]:
len(df[df['group']==1])

1031

In [85]:
df[df['group']==1]['fea']

2360    0.5966
2361    0.6234
2362    0.6404
2363    0.6582
2364    0.5916
         ...  
3386    0.6566
3387    0.6410
3388    0.6524
3389    0.5580
3390    0.6362
Name: fea, Length: 1031, dtype: float64